# 함수 사용 체인 

from messages와 from template 차이는?

In [3]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """
            너는 고객의 문의에 대응되는 주문 상태를 판단하는 로봇이야.
            사용자가 입력한 메시지를 보고 아래 주문 상태 중 하나로 분류해야 해.
            -주문 상품에 관한 문의: '주문 완료'
            -주문 변경에 관한 문의: '주문 변경'
            -주문 취소에 관한 문의: '주문 취소'
            """
        ),
        ("human", "{input}"),

    ]
)

In [2]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI()
model

ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x00000243F82DF760>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x00000243F82DE560>, openai_api_key=SecretStr('**********'), openai_proxy='')

In [4]:
chain = prompt | model
chain

ChatPromptTemplate(input_variables=['input'], messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template="\n            너는 고객의 문의에 대응되는 주문 상태를 판단하는 로봇이야.\n            사용자가 입력한 메시지를 보고 아래 주문 상태 중 하나로 분류해야 해.\n            -주문 상품에 관한 문의: '주문 완료'\n            -주문 변경에 관한 문의: '주문 변경'\n            -주문 취소에 관한 문의: '주문 취소'\n            ")), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}'))])
| ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x00000243F82DF760>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x00000243F82DE560>, openai_api_key=SecretStr('**********'), openai_proxy='')

In [7]:
response = chain.invoke({"input": "주문 내역을 알고 싶어"})
response.content

'주문 완료'

In [8]:
response = chain.invoke({"input": "떡케익을 취소하고 싶어"})
response.content

'주문 취소'

In [15]:
classify_inquiry_chain = prompt | model

In [9]:
def get_completed_orders(dict):
    user_id = dict["user_id"]
    return f"사용자 {user_id}의 주문 완료된 주문내역 조회 완료"

def get_payment_completed_orders(dict):
    user_id = dict["user_id"]
    return f"사용자 {user_id}의입금 완료된 주문내역 조회 완료"

def get_changed_orders(dict):
    user_id = dict["user_id"]
    return f"사용자 {user_id}의주문 변경된 주문내역 조회 완료"

def get_canceled_orders(dict):
    user_id = dict["user_id"]
    return f"사용자 {user_id}의주문 취소된 주문내역 조회 완료"

In [10]:
from langchain_core.runnables import RunnableLambda

RunnableLambda(get_canceled_orders).invoke({"uesr_id": 1})

'주문 취소된 주문내역 조회 완료'

In [21]:
def route(info):
    print("함수로 전달된 데이터 -> ", info)
    if "주문 변경" in info["inquiry_type"].lower():
        return RunnableLambda(get_changed_orders)
    elif "주문 취소" in info["inquiry_type"].lower():
        return RunnableLambda(get_canceled_orders)
    else:
        return RunnableLambda(get_completed_orders)

In [16]:
from langchain_core.runnables import RunnablePassthrough
chain1 = {"input": RunnablePassthrough(), "user_id": RunnablePassthrough(), "inquiry_type": classify_inquiry_chain} | route
chain1

TypeError: unsupported operand type(s) for |: 'dict' and 'function'

In [22]:
from langchain_core.runnables import RunnablePassthrough
chain1 = {"input": RunnablePassthrough(), "user_id": RunnablePassthrough(), "inquiry_type": classify_inquiry_chain} | RunnableLambda(route)
chain1

{
  input: RunnablePassthrough(),
  user_id: RunnablePassthrough(),
  inquiry_type: ChatPromptTemplate(input_variables=['input'], messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template="\n            너는 고객의 문의에 대응되는 주문 상태를 판단하는 로봇이야.\n            사용자가 입력한 메시지를 보고 아래 주문 상태 중 하나로 분류해야 해.\n            -주문 상품에 관한 문의: '주문 완료'\n            -주문 변경에 관한 문의: '주문 변경'\n            -주문 취소에 관한 문의: '주문 취소'\n            ")), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}'))])
                | ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x00000243F82DF760>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x00000243F82DE560>, openai_api_key=SecretStr('**********'), openai_proxy='')
}
| RunnableLambda(route)

In [23]:
chain1.invoke(
    {"input": "주문 내역을 보고 싶어",
     "user_id": 1,}
)

함수로 전달된 데이터 ->  {'input': {'input': '주문 내역을 보고 싶어', 'user_id': 1}, 'user_id': {'input': '주문 내역을 보고 싶어', 'user_id': 1}, 'inquiry_type': AIMessage(content='주문 완료', response_metadata={'token_usage': {'completion_tokens': 5, 'prompt_tokens': 144, 'total_tokens': 149}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-727cc11d-c783-49ba-ad3c-87eb8425e12d-0')}


AttributeError: 'AIMessage' object has no attribute 'lower'

In [24]:
def route(info):
    print("함수로 전달된 데이터 -> ", info)
    if "주문 변경" in info["inquiry_type"].content.lower():
        return RunnableLambda(get_changed_orders)
    elif "주문 취소" in info["inquiry_type"].content.lower():
        return RunnableLambda(get_canceled_orders)
    else:
        return RunnableLambda(get_completed_orders)

In [26]:
from langchain_core.runnables import RunnablePassthrough
chain2 = {"input": RunnablePassthrough(), "user_id": RunnablePassthrough(), "inquiry_type": classify_inquiry_chain} | RunnableLambda(route)
chain2

{
  input: RunnablePassthrough(),
  user_id: RunnablePassthrough(),
  inquiry_type: ChatPromptTemplate(input_variables=['input'], messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template="\n            너는 고객의 문의에 대응되는 주문 상태를 판단하는 로봇이야.\n            사용자가 입력한 메시지를 보고 아래 주문 상태 중 하나로 분류해야 해.\n            -주문 상품에 관한 문의: '주문 완료'\n            -주문 변경에 관한 문의: '주문 변경'\n            -주문 취소에 관한 문의: '주문 취소'\n            ")), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}'))])
                | ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x00000243F82DF760>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x00000243F82DE560>, openai_api_key=SecretStr('**********'), openai_proxy='')
}
| RunnableLambda(route)

In [27]:
response = chain2.invoke(
    {"input": "주문 내역을 보고 싶어"},
    {"user_id": 1}

)
response

함수로 전달된 데이터 ->  {'input': {'input': '주문 내역을 보고 싶어'}, 'user_id': {'input': '주문 내역을 보고 싶어'}, 'inquiry_type': AIMessage(content='주문 완료', response_metadata={'token_usage': {'completion_tokens': 5, 'prompt_tokens': 144, 'total_tokens': 149}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-01a5f6f8-cdce-4e93-a8ea-8ce9e94cbe6b-0')}


'주문 완료된 주문내역 조회 완료'

In [28]:
response = chain2.invoke(
    {"input": "주문 취소 정보를 보고 싶어"},
    {"user_id": 1}

)
response

함수로 전달된 데이터 ->  {'input': {'input': '주문 취소 정보를 보고 싶어'}, 'user_id': {'input': '주문 취소 정보를 보고 싶어'}, 'inquiry_type': AIMessage(content="주문 취소와 관련된 정보를 확인하고 싶으신 거군요! 주문 취소에 관한 문의를 하시는 거군요. 주문 상태는 '주문 취소'에 속합니다. 추가로 궁금한 점이 있으시면 언제든지 물어봐 주세요.", response_metadata={'token_usage': {'completion_tokens': 92, 'prompt_tokens': 145, 'total_tokens': 237}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-22bde337-ff3f-495e-8450-2346d9316e1e-0')}


'주문 취소된 주문내역 조회 완료'

In [30]:
response = chain2.invoke(
    {"input": "주문 취소 정보를 보고 싶어",
     "user_id": 1}
)
response

함수로 전달된 데이터 ->  {'input': {'input': '주문 취소 정보를 보고 싶어', 'user_id': 1}, 'user_id': {'input': '주문 취소 정보를 보고 싶어', 'user_id': 1}, 'inquiry_type': AIMessage(content='주문 취소', response_metadata={'token_usage': {'completion_tokens': 5, 'prompt_tokens': 145, 'total_tokens': 150}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-c6403f17-9bac-46af-9578-e2c8f90255bc-0')}


'주문 취소된 주문내역 조회 완료'

In [34]:
def get_completed_orders(dict):
    print(dict)
    user_id = dict["user_id"]
    return f"사용자 {user_id}의 주문 완료된 주문내역 조회 완료"

def get_payment_completed_orders(dict):
    print(dict)
    user_id = dict["user_id"]
    return f"사용자 {user_id}의입금 완료된 주문내역 조회 완료"

def get_changed_orders(dict):
    print(dict)
    user_id = dict["user_id"]
    return f"사용자 {user_id}의주문 변경된 주문내역 조회 완료"

def get_canceled_orders(dict):
    print(dict)
    user_id = dict["user_id"]
    return f"사용자 {user_id}의주문 취소된 주문내역 조회 완료"

In [35]:
def route(info):
    print("함수로 전달된 데이터 -> ", info)
    if "주문 변경" in info["inquiry_type"].content.lower():
        return RunnableLambda(get_changed_orders)
    elif "주문 취소" in info["inquiry_type"].content.lower():
        return RunnableLambda(get_canceled_orders)
    else:
        return RunnableLambda(get_completed_orders)

In [36]:
from langchain_core.runnables import RunnablePassthrough
chain3 = {"input": RunnablePassthrough(), "user_id": lambda x: x["user_id"], "inquiry_type": classify_inquiry_chain} | RunnableLambda(route)
chain3

{
  input: RunnablePassthrough(),
  user_id: RunnableLambda(lambda x: x['user_id']),
  inquiry_type: ChatPromptTemplate(input_variables=['input'], messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template="\n            너는 고객의 문의에 대응되는 주문 상태를 판단하는 로봇이야.\n            사용자가 입력한 메시지를 보고 아래 주문 상태 중 하나로 분류해야 해.\n            -주문 상품에 관한 문의: '주문 완료'\n            -주문 변경에 관한 문의: '주문 변경'\n            -주문 취소에 관한 문의: '주문 취소'\n            ")), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}'))])
                | ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x00000243F82DF760>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x00000243F82DE560>, openai_api_key=SecretStr('**********'), openai_proxy='')
}
| RunnableLambda(route)

In [37]:
response = chain3.invoke(
    {"input": "주문 취소 정보를 보고 싶어",
     "user_id": 1}
)
response

함수로 전달된 데이터 ->  {'input': {'input': '주문 취소 정보를 보고 싶어', 'user_id': 1}, 'user_id': 1, 'inquiry_type': AIMessage(content="주문 상태가 '주문 취소'에 해당하는 내용을 보고 싶다는 거죠?\n그렇다면 주문 취소에 관한 정보를 안내해 드릴게요. 혹시 추가로 궁금한 점이 있으신가요?", response_metadata={'token_usage': {'completion_tokens': 75, 'prompt_tokens': 145, 'total_tokens': 220}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-46f8f7d1-e654-4579-9035-9cf17a9059c5-0')}
{'input': {'input': '주문 취소 정보를 보고 싶어', 'user_id': 1}, 'user_id': 1, 'inquiry_type': AIMessage(content="주문 상태가 '주문 취소'에 해당하는 내용을 보고 싶다는 거죠?\n그렇다면 주문 취소에 관한 정보를 안내해 드릴게요. 혹시 추가로 궁금한 점이 있으신가요?", response_metadata={'token_usage': {'completion_tokens': 75, 'prompt_tokens': 145, 'total_tokens': 220}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-46f8f7d1-e654-4579-9035-9cf17a9059c5-0')}


'사용자 1의주문 취소된 주문내역 조회 완료'

# 문의/요청 분류 체인

In [38]:
from langchain_core.prompts import ChatPromptTemplate

message_type_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """
            너는 고객 입력 메시지를 아래 두 유형 중 하나로 분류하는 로봇이야.
            -상품 문의, 주문 내역 조회, 주문 변경 내역 조회, 주문 취소 내역 조회: '문의'
            -주문 요청, 주문 변경 요청, 주문 취소 요청: '요청'
            """
        ),
        ("human", "{input}"),

    ]
)

In [39]:
classify_message_chain = message_type_prompt | model 
classify_message_chain

ChatPromptTemplate(input_variables=['input'], messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template="\n            너는 고객 입력 메시지를 아래 두 유형 중 하나로 분류하는 로봇이야.\n            -상품 문의, 주문 내역 조회, 주문 변경 내역 조회, 주문 취소 내역 조회: '문의'\n            -주문 요청, 주문 변경 요청, 주문 취소 요청: '요청'\n            ")), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}'))])
| ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x00000243F82DF760>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x00000243F82DE560>, openai_api_key=SecretStr('**********'), openai_proxy='')

In [40]:
response = classify_message_chain.invoke("판매 제품 좀 알려줘")
response

AIMessage(content='문의', response_metadata={'token_usage': {'completion_tokens': 2, 'prompt_tokens': 132, 'total_tokens': 134}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-a6fbf0de-1be0-449c-8081-6f205a58d9c4-0')

In [41]:
handle_inquiry_chain = {"input": RunnablePassthrough(), "user_id": lambda x: x["user_id"], "inquiry_type": classify_inquiry_chain} | RunnableLambda(route)
handle_inquiry_chain

{
  input: RunnablePassthrough(),
  user_id: RunnableLambda(lambda x: x['user_id']),
  inquiry_type: ChatPromptTemplate(input_variables=['input'], messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template="\n            너는 고객의 문의에 대응되는 주문 상태를 판단하는 로봇이야.\n            사용자가 입력한 메시지를 보고 아래 주문 상태 중 하나로 분류해야 해.\n            -주문 상품에 관한 문의: '주문 완료'\n            -주문 변경에 관한 문의: '주문 변경'\n            -주문 취소에 관한 문의: '주문 취소'\n            ")), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}'))])
                | ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x00000243F82DF760>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x00000243F82DE560>, openai_api_key=SecretStr('**********'), openai_proxy='')
}
| RunnableLambda(route)

In [42]:
def inquiry_request_route(info):
    if "문의" in info["inquiry_type"].content.lower():
        return handle_inquiry_chain
    else:
        return RunnableLambda(get_completed_orders)

In [43]:
chain4 = classify_message_chain | inquiry_request_route
chain4.invoke("주문 내역을 알고 싶어")

TypeError: 'AIMessage' object is not subscriptable

In [44]:
def inquiry_request_route(info):
    print(info)
    if "문의" in info["inquiry_type"].content.lower():
        return handle_inquiry_chain
    else:
        return RunnableLambda(get_completed_orders)

In [45]:
chain4 = classify_message_chain | inquiry_request_route
chain4.invoke("주문 내역을 알고 싶어")

content='문의' response_metadata={'token_usage': {'completion_tokens': 2, 'prompt_tokens': 128, 'total_tokens': 130}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None} id='run-12d905c8-0d67-4f3f-929d-3034b285539d-0'


TypeError: 'AIMessage' object is not subscriptable

In [46]:
from langchain_core.messages import AIMessage

def inquiry_request_route(message: AIMessage):
    if "문의" in message.content.lower():
        return handle_inquiry_chain
    else:
        return RunnableLambda(get_completed_orders)

아마 handle_inquiry_chain에 전달되는 게 그 전 체인이 출력하는  response_metadata={'token_usage': {'completion_tokens': 2, 같은 AI Message인 듯

In [47]:
chain4 = classify_message_chain | inquiry_request_route
chain4.invoke("주문 내역을 알고 싶어")

TypeError: 'AIMessage' object is not subscriptable

In [48]:
from langchain_core.messages import AIMessage

def inquiry_request_route(dict):
    print(dict)
    if "문의" in dict["msg_type"].content.lower():
        return handle_inquiry_chain
    else:
        return RunnableLambda(get_completed_orders)

따로 dict 키 등에 할당하지 않으면 그냥 출력 결과만 전달되네

In [50]:
chain4 = classify_message_chain | inquiry_request_route
chain4.invoke(
    {"input": "주문 내역을 알고 싶어",
     "user_id": 3}
     )

content='문의' response_metadata={'token_usage': {'completion_tokens': 2, 'prompt_tokens': 128, 'total_tokens': 130}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None} id='run-cbed123c-52ff-49c1-aeca-ec219c7fef83-0'


TypeError: 'AIMessage' object is not subscriptable

In [52]:
classify_message_chain = {"input": RunnablePassthrough(),
                          "user_id": RunnablePassthrough(),
                          "msg_type": message_type_prompt | model} 
classify_message_chain

{'input': RunnablePassthrough(),
 'user_id': RunnablePassthrough(),
 'msg_type': ChatPromptTemplate(input_variables=['input'], messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template="\n            너는 고객 입력 메시지를 아래 두 유형 중 하나로 분류하는 로봇이야.\n            -상품 문의, 주문 내역 조회, 주문 변경 내역 조회, 주문 취소 내역 조회: '문의'\n            -주문 요청, 주문 변경 요청, 주문 취소 요청: '요청'\n            ")), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}'))])
 | ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x00000243F82DF760>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x00000243F82DE560>, openai_api_key=SecretStr('**********'), openai_proxy='')}

In [53]:
classify_message_chain.invoke(
    {"input": "주문 내역을 알고 싶어",
     "user_id": 3}
)

AttributeError: 'dict' object has no attribute 'invoke'

In [ ]:
chain4 = classify_message_chain | inquiry_request_route
chain4.invoke(
    
     )

In [ ]:
from langchain_core.runnables import RunnableParallel

RunnableParallel(
    
)

In [54]:
classify_message_chain = RunnablePassthrough.assign(msg_type=message_type_prompt | model)
classify_message_chain

RunnableAssign(mapper={
  msg_type: ChatPromptTemplate(input_variables=['input'], messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template="\n            너는 고객 입력 메시지를 아래 두 유형 중 하나로 분류하는 로봇이야.\n            -상품 문의, 주문 내역 조회, 주문 변경 내역 조회, 주문 취소 내역 조회: '문의'\n            -주문 요청, 주문 변경 요청, 주문 취소 요청: '요청'\n            ")), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}'))])
            | ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x00000243F82DF760>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x00000243F82DE560>, openai_api_key=SecretStr('**********'), openai_proxy='')
})

In [55]:
classify_message_chain.invoke(
    {"input": "주문 내역을 알고 싶어",
     "user_id": 3}
)

{'input': '주문 내역을 알고 싶어',
 'user_id': 3,
 'msg_type': AIMessage(content='문의', response_metadata={'token_usage': {'completion_tokens': 2, 'prompt_tokens': 128, 'total_tokens': 130}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-9806c226-3301-4744-bda4-417433b50ff1-0')}

In [56]:
chain5 = classify_message_chain | inquiry_request_route
chain5

RunnableAssign(mapper={
  msg_type: ChatPromptTemplate(input_variables=['input'], messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template="\n            너는 고객 입력 메시지를 아래 두 유형 중 하나로 분류하는 로봇이야.\n            -상품 문의, 주문 내역 조회, 주문 변경 내역 조회, 주문 취소 내역 조회: '문의'\n            -주문 요청, 주문 변경 요청, 주문 취소 요청: '요청'\n            ")), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}'))])
            | ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x00000243F82DF760>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x00000243F82DE560>, openai_api_key=SecretStr('**********'), openai_proxy='')
})
| RunnableLambda(inquiry_request_route)

In [57]:
chain5.invoke(
    {"input": "주문 내역을 알고 싶어",
     "user_id": 3}
)

{'input': '주문 내역을 알고 싶어', 'user_id': 3, 'msg_type': AIMessage(content='문의', response_metadata={'token_usage': {'completion_tokens': 2, 'prompt_tokens': 128, 'total_tokens': 130}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-ea7b5a28-2583-4440-91ba-24d1fea1f8c4-0')}
함수로 전달된 데이터 ->  {'input': {'input': '주문 내역을 알고 싶어', 'user_id': 3, 'msg_type': AIMessage(content='문의', response_metadata={'token_usage': {'completion_tokens': 2, 'prompt_tokens': 128, 'total_tokens': 130}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-ea7b5a28-2583-4440-91ba-24d1fea1f8c4-0')}, 'user_id': 3, 'inquiry_type': AIMessage(content='주문 완료', response_metadata={'token_usage': {'completion_tokens': 5, 'prompt_tokens': 145, 'total_tokens': 150}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-3c5e2d58-0997-40be-b55b-802effca7802-0

'사용자 3의 주문 완료된 주문내역 조회 완료'

In [58]:
chain5.invoke(
    {"input": "떡케익 주문하고 싶어요",
     "user_id": 3}
)

{'input': '떡케익 주문하고 싶어요', 'user_id': 3, 'msg_type': AIMessage(content='요청', response_metadata={'token_usage': {'completion_tokens': 3, 'prompt_tokens': 132, 'total_tokens': 135}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-4118ff99-358b-44f1-90e4-875b6516e47a-0')}
{'input': '떡케익 주문하고 싶어요', 'user_id': 3, 'msg_type': AIMessage(content='요청', response_metadata={'token_usage': {'completion_tokens': 3, 'prompt_tokens': 132, 'total_tokens': 135}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-4118ff99-358b-44f1-90e4-875b6516e47a-0')}


'사용자 3의 주문 완료된 주문내역 조회 완료'

# 메모리 추가

In [ ]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

initial_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "너는 주문봇이야"
            "이전 대화와 현재 고객이 입력한 메시지 모두를 꼼꼼히 파악하여 답변해줘."
        ),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "사용자 ID: {user_id}\n사용자 입력 메시지{input}"),
    ]
)

In [67]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

message_type_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """
            너는 고객 입력 메시지를 아래 두 유형 중 하나로 분류하는 로봇이야.
            -상품 문의, 주문 내역 조회, 주문 변경 내역 조회, 주문 취소 내역 조회: '문의'
            -주문 요청, 주문 변경 요청, 주문 취소 요청: '요청'
            """
        ),
        MessagesPlaceholder(variable_name="chat_history")
        ("human", "{input}"),

    ]
)

TypeError: 'MessagesPlaceholder' object is not callable

In [61]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

inquiry_type_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """
            너는 고객의 문의에 대응되는 주문 상태를 판단하는 로봇이야.
            사용자가 입력한 메시지를 보고 아래 주문 상태 중 하나로 분류해야 해.
            -주문 상품에 관한 문의: '주문 완료'
            -주문 변경에 관한 문의: '주문 변경'
            -주문 취소에 관한 문의: '주문 취소'
            """
        ),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "{input}"),

    ]
)

In [59]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory

store = {}

def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

In [72]:
classify_message_chain = message_type_prompt | model

In [73]:
from langchain_core.runnables.history import RunnableWithMessageHistory

classify_message_with_memory = RunnableWithMessageHistory(
    classify_message_chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history",
)

In [74]:
classify_message_with_memory.invoke(
    {"input": "주문 상태가 '주문 완료'인 주문을 알려줘"},
    config={"configurable": {"session_id": "test_240514-1"}}
    )

Parent run f91ef360-abc4-485b-97c8-2b86f77ec48f not found for run 30eae7c8-7cae-4279-83de-0be5daea3103. Treating as a root run.


AIMessage(content='문의', response_metadata={'token_usage': {'completion_tokens': 2, 'prompt_tokens': 138, 'total_tokens': 140}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-2772ef57-d2e6-4e86-b52f-b6d39bf26da1-0')

In [66]:
store["test_240514-1"]

InMemoryChatMessageHistory(messages=[HumanMessage(content="주문 상태가 '주문 완료'인 주문을 알려줘"), AIMessage(content="주문 상태가 '주문 완료'인 주문은 다음과 같습니다.", response_metadata={'token_usage': {'completion_tokens': 22, 'prompt_tokens': 155, 'total_tokens': 177}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-992fb359-6642-464f-ab56-339567e4b320-0')])

In [70]:
del store["test_240514-1"]

In [71]:
store

{}

In [ ]:
def inquiry_request_route(info):
    print(info)
    if "문의" in info["inquiry_type"].content.lower():
        return handle_inquiry_chain
    else:
        return RunnableLambda(get_completed_orders)

In [ ]:
handle_inquiry_chain = {"input": RunnablePassthrough(), "user_id": lambda x: x["user_id"], "inquiry_type": classify_inquiry_chain} | RunnableLambda(route)
handle_inquiry_chain

In [ ]:
chain